# This notebook will demonstrate the spectrum of Snowflake privacy and data sharing capabilities

## Consumer Account

In [ ]:
-- Set these to the account locators of your Provider and Consumer accounts respectively
set PROVIDERACCOUNT= 'XXX12345';
set CONSUMERACCOUNT= 'YYY98765';
set PROVIDERSHARE=$PROVIDERACCOUNT||'.customer_share';

## Setup environment and seed test data
## Provide must share customer table before running this step

In [ ]:
use role ACCOUNTADMIN; 
create or replace database customerdb_local; 
create or replace schema customerdb_local.globalsales;

use schema customerdb_local.globalsales;

### Load test data from Snowsight UI

After downloading the *.csv from GitHub, execute the following steps: 

1. Under the **Data** tab in the left menu, Click **Databases**
2. Click database **CUSTOMERDB_LOCAL** and schema **GLOBALSALES**
3. In the upper right, click the **Create** dropdown and select **Table** then **From File**
4. Drag your *.CSV (customer_consumer.csv) to the center of the screen and drop. You can alternatively click **Browse** and locate your file
5. On the **Load Data into Table** screen, go to the bottom. Keep "Create New Table" selected. Enter "customer" under the **Name** field and click **Next**
6. This will take you to a panel that lists all columnms. Leave all default values as-is and click **Load**. 

You should see a message that 399 rows were successfully loaded. 


## Demo Starts Here

In [ ]:
-- set context
use role accountadmin;
use schema customerdb_local.globalsales;

In [ ]:
-- Look at my local data
select * from customerdb_local.globalsales.customer;

## Lets create a database from what the Provider shared
## This step can also be performed in the UI

In [ ]:
create or replace database customerdb_provider
from share identifier($PROVIDERSHARE);

## Lets do a select from the customer table that was shared with us

In [ ]:
select * 
from customerdb_provider.globalsales.customer 
limit 10;

### Check that we can see the View that just got added to the share by the provider

In [ ]:
select * 
from customerdb_provider.globalsales.customerV;

### Now back to the provider to try some privacy policies

In [ ]:
select * 
from customerdb_provider.globalsales.customer;

## That failed on a projection policy

## Lets try avoiding those columns

In [ ]:
select c_custkey, c_gender, c_age, c_acctbal 
from customerdb_provider.globalsales.customer;

### That failed on the aggregation policy
### so lets try an aggregate query

### This query works but the results are very granular - all groups have < 5 rows so they get filtered from the results

In [ ]:
select c_custkey, c_gender, c_age, sum(c_acctbal)
from customerdb_provider.globalsales.customer
group by 1,2,3
order by 1,2,3;

## Must be fewer than 5 rows in each group. Lets eliminate the custkey

In [ ]:
select c_gender, c_age, sum(c_acctbal), count(*)
from customerdb_provider.globalsales.customer
group by 1,2
order by 4;

## Now lets join it to our own (local) data set to get the balance on my customers using the demographics from the provider data

### I can use c_email because its not in the projection!!!

In [ ]:
select a.c_age, a.c_gender, count(*), sum(b.c_acctbal)
from 
    customerdb_provider.globalsales.customer a 
    inner join
    customerdb_local.globalsales.customer b
    on a.c_email = b.c_email
group by 1,2
order by 1,2; 

## Get the total balances of my customer who are NOT also customers of the provider

In [ ]:
select sum(c_acctbal) 
from customerdb_local.globalsales.customer
where c_email not in 
(select c_email 
 from customerdb_provider.globalsales.customer);

### Validate the Differential Privacy Policy

In [ ]:
SELECT COUNT(DISTINCT c_custkey) AS c,
       DP_INTERVAL_LOW(c) AS low,
       DP_INTERVAL_HIGH(c) AS high,
FROM customerdb_provider.globalsales.customer
WHERE c_gender='F'
  AND c_age between 30 and 50;